
#  "ANÁLISE DO IMPACTO CAUSAL DA LEI Nº 12.034/2009 EM ELEIÇÕES BRASILEIRAS"

---------

###   Objetivo

**Este notebook tem como objetivo somente preparar o dataframe para análises**

Este estudo avalia se a Lei nº 12.034/2009, que exige 30% de candidaturas femininas, aumentou a particpação de mulheres para, vereadora, deputada estadual e distrital. Usando dados do TSE e modelos econométricos de inferência causal, compara o desempenho de mulheres antes e depois da lei. A contribuição central é mostrar evidências sobre os efeitos da cota e discutir a possível ampliação para eleições majoritárias.

### O que é a "Minirreforma Eleitoral" de 2009?

A "minirreforma eleitoral" de 2009, estabelecida pela Lei 12.034/2009, alterou diversas leis eleitorais brasileiras. O ponto mais importante dela é a mudança na regra das **cotas de gênero para candidaturas proporcionais**.

---------

### A regra da cota de gênero (Art. 10, §3º da Lei das Eleições)

Desde 2009, a lei exige que nas eleições proporcionais (deputado federal, estadual/distrital e vereador), cada partido ou federação preencha um mínimo de **30%** e um máximo de **70%** de candidaturas para cada gênero.

Antes da reforma, a lei usava o termo "reservar" vagas. A mudança para "preencher" eliminou a brecha que permitia aos partidos apenas separar as vagas no papel sem efetivamente lançar candidatas.

---------

### Escopo da lei

* **Aplica-se a:** eleições proporcionais (deputado federal, estadual/distrital e vereador).
* **Não se aplica a:** eleições majoritárias (presidente, governador, prefeito, senador).

---------

### Como a Justiça Eleitoral fiscaliza e pune

A verificação do cumprimento da cota é feita por **partido ou federação**. Se a regra não for seguida, o **Demonstrativo de Regularidade de Atos Partidários (DRAP)** pode ser indeferido, prejudicando todos os registros individuais de candidaturas da chapa.

Quando há **fraude à cota** (as chamadas "candidaturas laranja"), como candidatas que não recebem votos, não têm campanha ou financiamento, a Justiça Eleitoral pode tomar as seguintes medidas, conforme a Súmula 73 do Tribunal Superior Eleitoral (TSE):

* Cassação do DRAP e dos diplomas dos eleitos.
* Anulação dos votos de toda a chapa, com recálculo dos quocientes eleitorais.
* Declaração de inelegibilidade para os responsáveis pela fraude.

---------

### Observação adicional

O Tribunal Superior Eleitoral (TSE) e o Supremo Tribunal Federal (STF) também vincularam a distribuição do **fundo eleitoral** e do **tempo de rádio/TV** às mesmas proporções de gênero, exigindo que no mínimo 30% desses recursos sejam destinados às candidaturas femininas, caso o partido tenha o mínimo de 30% de candidatas.

---------

### Dados google BigQuery da Base dos dados (BD)
https://console.cloud.google.com/bigquery?p=basedosdados&d=br_tse_eleicoes&t=candidatos&page=table&inv=1&invt=Ab5S-w&project=argon-magnet-465219-g1&ws=!1m5!1m4!4m3!1sbasedosdados!2sbr_tse_eleicoes!3scandidatos

### Importar bibliotecas

In [ ]:
# %pip install pyspark

In [ ]:
# Importar bibliotecas
from pyspark.sql.functions import (
    when,                   # expressão condicional (equivalente a CASE WHEN)
    lit,                    # cria um literal/constante como coluna
    regexp_replace,         # substitui texto usando expressão regular (regex)
    lower,                  # converte strings para minúsculas
    split,                  # divide string em array pelo separador
    trim,                   # remove espaços no início e no fim da string
    col,                    # referencia uma coluna pelo nome
    coalesce,               # primeiro valor não nulo entre várias colunas
    isnan,                  # verifica NaN (numéricos); não confundir com NULL
    countDistinct,          # conta valores distintos
    dense_rank,             # ranking “sem buracos” (usa Window para ordenar)
    expr,                   # escreve expressões SQL diretamente
    last,
    col,
    count as spark_count,   # contagem de linhas (agg)
    mean as spark_mean,     # média (agg)
    min as spark_min,       # mínimo (agg)
    max as spark_max,       # máximo (agg)
    stddev as spark_sd,     # desvio-padrão amostral (agg)
    sum as spark_sum        # soma (agg)
)

from pyspark.sql.window import Window     # Define "janelas" para cálculos sobre partições (rank, row_number, etc.)
from pyspark.sql.types import StringType  # Tipo de dado string para schemas/colunas em DataFrames Spark
import plotly.express as px               # Biblioteca de visualização rápida e simples (gráficos interativos)
from pyspark.sql import functions as F    # Importa funções SQL do Spark com alias F (ex: F.col, F.sum)
from plotly.subplots import make_subplots # Cria layouts com múltiplos gráficos/plots no Plotly
import plotly.graph_objects as go         # Criação detalhada de gráficos com objetos (mais flexível que px)
import pandas as pd                       # Biblioteca para manipulação de dados em DataFrames no Python
from pyspark.sql.types import IntegerType # Importa conversor de coluna pára tipo interger
import numpy as np
from math import isnan
from scipy.stats import norm
from IPython.display import HTML, display
#%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Importar dados

In [0]:
# Importar tabela dos candidatos eleitos - TSE
df_candidatos_eleitos = spark.sql("SELECT * FROM workspace.avaliacao_lei_tse.candidatos_eleitos")

# Importar tabela dos candidatos - TSE
df_candidatos = spark.sql("SELECT * FROM workspace.avaliacao_lei_tse.candidatos")

# Importar tabela de genero faltantes
missing_gender_human_input = spark.sql("SELECT * FROM workspace.avaliacao_lei_tse.missing_gender_human_input")

# Importar tabela das variáveis explicativas 
df_explicativas = spark.sql("SELECT * FROM workspace.avaliacao_lei_tse.explicativas")

# visualizar
# display(df_explicativas, df_candidatos, df_candidatos_eleitos, missing_gender_human_input)


### Filtrar df dos candidatos eleitos e fazer join com o df candidatos

In [0]:
# Renomear cedo para evitar o uso de crase em colunas com espaço
if "eleitores total" in df_explicativas.columns:
    df_explicativas = df_explicativas.withColumnRenamed("eleitores total", "eleitores_total")

# Eleitos: chave robusta e join
df_eleitos = (
    df_candidatos_eleitos
    .filter((col("ano") >= 1992) & col("resultado").rlike("(?i)^eleit"))
    .select(
        col("ano").cast("int").alias("ano"),
        F.upper(F.trim(col("sigla_uf"))).alias("sigla_uf_eleito"),
        trim(col("cargo")).alias("cargo_eleito"),
        trim(col("titulo_eleitoral_candidato")).alias("titulo_eleitoral"),
        lit("eleito").alias("resultado"),
        col("votos")
    )
    .dropDuplicates(["titulo_eleitoral", "ano", "sigla_uf_eleito", "cargo_eleito"])
)

df_candidatos = (
    df_candidatos
    .withColumn("ano", col("ano").cast("int"))
    .withColumn("sigla_uf", F.upper(F.trim(F.regexp_replace(F.col("sigla_uf"), "\u00A0", " "))))
    .withColumn("sigla_uf_nascimento", F.upper(F.trim(F.regexp_replace(F.col("sigla_uf_nascimento"), "\u00A0", " "))))
    .withColumn("titulo_eleitoral", trim(col("titulo_eleitoral")))
)

# join pelo par (ano, titulo_eleitoral) — chave prática no universo TSE
df_candidatos = (
    df_candidatos
    .join(df_eleitos.select("ano", "titulo_eleitoral", "resultado"), on=["ano","titulo_eleitoral"], how="left")
    .fillna({"resultado": "nao_eleito"})
)

# Mostra a contagem de eleitos x não eleitos
display(df_candidatos.groupBy("resultado").count())

DataFrame[resultado: string, count: bigint]

---------

### Correção dos valores ausentes no campo `genero`

- Na eleição de **1996**, observou-se que **100%** dos registros de candidatos estavam sem informação de gênero.
- Para corrigir esses valores, adotamos uma estratégia de **propagação por primeiro nome**:

  1. **Gerar amostra base:** criar um conjunto auxiliar contendo apenas o **primeiro nome** do candidato e seu gênero, considerando somente registros onde o gênero está preenchido.
  2. **Remover duplicados:** manter apenas combinações únicas de primeiro nome e gênero.
  3. **Aplicar preenchimento:** realizar um `join` entre a base original e essa amostra pela chave *primeiro nome*. Assim, por exemplo, uma candidata chamada **Maria** identificada como feminina em 2000 fornecerá essa informação para preencher o gênero de outras "Marias" que estejam com o campo ausente.
- Esse método permite reduzir drasticamente os valores faltantes, aproveitando padrões consistentes entre nome e gênero.


### Inferir gênero faltante a partir do primeiro nome

In [0]:
df_filtrado = (
    df_candidatos
    .withColumn("primeiro_nome", lower(split(trim(col("nome")), " ").getItem(0)))
    .withColumn("genero_l", lower(col("genero")))
    .filter(
        (col("primeiro_nome").isNotNull()) & (col("primeiro_nome") != "") &
        (col("genero_l").isNotNull()) & (col("genero_l") != "")
    )
)

df_contagem = (
    df_filtrado.groupBy("primeiro_nome", "genero_l")
    .agg(spark_count("*").alias("n"))
    .withColumn("p", col("n") / spark_sum("n").over(Window.partitionBy("primeiro_nome")))
)

df_top = (
    df_contagem
    .withColumn("rank", expr("row_number() over (partition by primeiro_nome order by p desc)"))
    .filter(col("rank") == 1)
)

amostra_genero = df_top.select("primeiro_nome", col("genero_l").alias("genero_inferido"))

# visualizar
display(amostra_genero)

DataFrame[primeiro_nome: string, genero_inferido: string]

In [0]:
# Imputar por primeiro nome
df_genero_fix = (
    df_candidatos
    .withColumn("primeiro_nome", lower(split(trim(col("nome")), " ").getItem(0)))
    .join(amostra_genero, on="primeiro_nome", how="left")
    .withColumn(
        "genero",
        when((col("genero").isNull()) | (trim(col("genero")) == ""), col("genero_inferido")).otherwise(col("genero"))
    )
    .drop("genero_inferido", "primeiro_nome")
)

# (Opcional) Aplicar input humano por nome completo
human_input = (
    missing_gender_human_input
    .select(
        trim(regexp_replace(col("nome"), "\u00A0", " ")).alias("nome_norm"),
        lower(col("genero_input_humano")).alias("genero_input_humano")
    )
)

df_genero_fix = (
    df_genero_fix
    .withColumn("nome_norm", trim(regexp_replace(col("nome"), "\u00A0", " ")))
    .join(human_input, on="nome_norm", how="left")
    .withColumn(
        "genero",
        when((col("genero").isNull()) | (trim(col("genero")) == ""), col("genero_input_humano")).otherwise(col("genero"))
    )
    .drop("genero_input_humano", "nome_norm")
)

# Checagem de faltantes de gênero
antes  = df_candidatos.filter((col("genero").isNull()) | (trim(col("genero")) == "")).count()
depois = df_genero_fix.filter((col("genero").isNull()) | (trim(col("genero")) == "")).count()
print({"faltantes_genero_antes": antes, "faltantes_genero_depois": depois})

# Remover dataframes auxiliares criados
del antes, depois, amostra_genero, df_contagem, df_filtrado, df_top, human_input, missing_gender_human_input

{'faltantes_genero_antes': 12537, 'faltantes_genero_depois': 0}


### Gerar coluna Regiões

In [0]:
# UF → Região + validações
mapeamento_regioes = {
    "AC":"Norte","AM":"Norte","AP":"Norte","PA":"Norte","RO":"Norte","RR":"Norte","TO":"Norte",
    "AL":"Nordeste","BA":"Nordeste","CE":"Nordeste","MA":"Nordeste","PB":"Nordeste",
    "PE":"Nordeste","PI":"Nordeste","RN":"Nordeste","SE":"Nordeste",
    "DF":"Centro-Oeste","GO":"Centro-Oeste","MT":"Centro-Oeste","MS":"Centro-Oeste",
    "ES":"Sudeste","MG":"Sudeste","RJ":"Sudeste","SP":"Sudeste",
    "PR":"Sul","RS":"Sul","SC":"Sul"
}
valid_ufs = set(mapeamento_regioes.keys())

# Preencher sigla_uf faltante com UF de nascimento
df_genero_fix = df_genero_fix.withColumn("sigla_uf", coalesce(col("sigla_uf"), col("sigla_uf_nascimento")))

# Validar UFs
ufs_ruins = df_genero_fix.filter(~col("sigla_uf").isin(list(valid_ufs))).select("sigla_uf").distinct()
if ufs_ruins.count() > 0:
    raise ValueError(f"UF(s) inesperada(s): {ufs_ruins.toPandas()['sigla_uf'].tolist()}")

mapping_expr = F.create_map([F.lit(x) for kv in mapeamento_regioes.items() for x in kv])
df_genero_uf_fix = df_genero_fix.withColumn("regioes", mapping_expr[col("sigla_uf")])


-----------------

### Filtrar candidaturas pela situação

**Remover (candidaturas não efetivas)**

* Essas situações não representam disputa real, logo não deveriam entrar no cálculo da cota ou da análise de desempenho eleitoral:
* indeferido / indeferido com recurso → não aparecem na urna.
* cassado / cassado com recurso → candidatura anulada.
* cancelado / cancelado com recurso → registro cancelado.
* inelegível → não pode concorrer.
* impugnado → barrado antes do pleito.
* renúncia → candidato desistiu.
* falecido / falecimento → óbvio, inviável.
* pedido não conhecido / pedido não conhecido com recurso / não conhecimento do pedido → não chegou a ser analisado no mérito.
* pendente de julgamento / substituto pendente de julgamento / substituto majoritário pendente de julgamento / aguardando julgamento → não chegaram a tempo de disputar.

* Isso garante que você está analisando candidaturas efetivamente válidas para a disputa proporcional

**NOTA IMPORTANTE** A coluna situação só tem dados apartir de 2002, portanto todas as candidaturas para anos menores que 2002 foram consideradas válidas


In [0]:
# Status da candidatura + cargo_std + eleito
# Nota: aqui trato NULL/vazio como INVÁLIDO
cond_valido = lower(trim(col("situacao"))).isin("deferido","sub judice","deferido com recurso")

df_candidatos_all = (
    df_genero_uf_fix
    .withColumn("status_candidatura", when(cond_valido, "valido").otherwise("invalido"))
)

df_candidatos_validos = (
    df_candidatos_all
    .filter(lower(col("status_candidatura")) == "valido")
    .withColumn(
        "cargo_std",
        when(col("cargo").rlike("(?i)deputado\\s*distrital"), lit("deputado distrital"))
        .when(col("cargo").rlike("(?i)deputado\\s*estadual"),  lit("deputado estadual"))
        .when(col("cargo").rlike("(?i)deputado\\s*federal"),   lit("deputado federal"))
        .when(col("cargo").rlike("(?i)vereador"),              lit("vereador"))
        .when(col("cargo").rlike("(?i)(senador|suplente.*senador|senador.*suplente)"), lit("senador"))
        .when(col("cargo").rlike("(?i)(governador|vice[- ]?governador)"),              lit("governador"))
        .when(col("cargo").rlike("(?i)(prefeit|vice[- ]?prefeit)"),                    lit("prefeito"))
        .otherwise(lit(None).cast("string"))
    )
    .withColumn("resultado_low", lower(col("resultado")))
    .withColumn("genero_low",    lower(col("genero")))
    .withColumn(
        "eleito",
        when((col("resultado_low").rlike("eleit")) & (~col("resultado_low").rlike("n(?:ão|ao)")), lit(1)).otherwise(lit(0))
    )
    .filter(col("cargo_std").isNotNull())
)

# Opcional: salvar base granular já saneada
(df_candidatos_validos
 .write.mode("overwrite").option("mergeSchema", "true")
 .saveAsTable("workspace.avaliacao_lei_tse.candidatos_genero_ok"))

### Montar data panel econometrics


In [0]:
cond_genero_valido = col("genero_low").isin("feminino","masculino")

painel_uf = (
    df_candidatos_validos
    .groupBy("sigla_uf","ano","cargo_std")
    .agg(
        F.sum(F.when(cond_genero_valido, 1).otherwise(0)).alias("n_total"),
        F.sum(F.when(col("genero_low")=="feminino", 1).otherwise(0)).alias("n_candidatas"),
        F.sum(F.when((col("eleito")==1) & cond_genero_valido, 1).otherwise(0)).alias("n_eleitos_total"),
        F.sum(F.when((col("eleito")==1) & (col("genero_low")=="feminino"), 1).otherwise(0)).alias("n_eleitas"),
    )
    .withColumn("prop_candidatas", F.when(col("n_total")>0, col("n_candidatas")/col("n_total")).otherwise(lit(None).cast("double")))
    .withColumn("prop_eleitas",    F.when(col("n_eleitos_total")>0, col("n_eleitas")/col("n_eleitos_total")).otherwise(lit(None).cast("double")))
    .withColumn("id_estado", col("sigla_uf"))
    .withColumn("id_cargo",  col("cargo_std"))
    .filter((col("ano")>=1998) & (col("ano")<=2022))
)


In [0]:
# --------------------------
# Painel balanceado com LOCF (uma vez só)
# --------------------------
ano_min = 1998
ano_max = 2022
anos_todos = list(range(ano_min, ano_max+1))
anos_df = spark.createDataFrame([(a,) for a in anos_todos], ["ano"])

chaves = ["sigla_uf","cargo_std"]

grid = painel_uf.select(*chaves).distinct().crossJoin(anos_df)

painel_uf_full = grid.join(painel_uf, on=chaves+["ano"], how="left")

# LOCF forward + backward somente nas colunas de proporção/contagem
w_fwd  = Window.partitionBy("sigla_uf","cargo_std").orderBy("ano").rowsBetween(Window.unboundedPreceding, 0)
w_back = Window.partitionBy("sigla_uf","cargo_std").orderBy(col("ano").desc()).rowsBetween(Window.unboundedPreceding, 0)

for c in ["n_total","n_candidatas","prop_candidatas","n_eleitos_total","n_eleitas","prop_eleitas"]:
    painel_uf_full = painel_uf_full.withColumn(c, last(col(c), ignorenulls=True).over(w_fwd))
    painel_uf_full = painel_uf_full.withColumn(c, coalesce(col(c), last(col(c), ignorenulls=True).over(w_back)))

# Flags tratado/post somente aqui (evita inconsistência)
def col_tratado(cargo_col):
    return F.when(cargo_col.isin("vereador","deputado federal","deputado estadual","deputado distrital"), lit(1)).otherwise(lit(0))

def col_treat_post(cargo_col, ano_col):
    return (
        F.when(cargo_col=="vereador", F.when(ano_col>=2012,1).otherwise(0))
         .when(cargo_col.isin("deputado federal","deputado estadual","deputado distrital"),
               F.when(ano_col>=2010,1).otherwise(0))
         .otherwise(0)
    )

painel_uf_bal = (
    painel_uf_full
    .withColumn("tratado",    col_tratado(col("cargo_std")))
    .withColumn("treat_post", col_treat_post(col("cargo_std"), col("ano")))
    .withColumn("id_estado",  col("sigla_uf"))
    .withColumn("id_cargo",   col("cargo_std"))
    .filter((col("ano")>=ano_min) & (col("ano")<=ano_max))
    .orderBy("sigla_uf","cargo_std","ano")
)

# # display(painel_uf_bal)

In [0]:
panel_data = painel_uf_bal.join(df_explicativas, on=["sigla_uf","ano"], how="left")

# proporção de eleitoras mulheres
panel_data = panel_data.withColumn("prop_eleitoras_mulheres", col("eleitoras_mulheres")/col("eleitores_total"))

# Função de log com clipping seguro (evita -inf/NaN)
def log_pos(colname, eps=1e-6):
    return F.log(F.when(F.col(colname) <= 0, eps).otherwise(F.col(colname)))

panel_data = (
    panel_data
    .withColumn("ln_prop_eleitas",            log_pos("prop_eleitas"))
    .withColumn("ln_prop_candidatas",         log_pos("prop_candidatas"))
    .withColumn("ln_populacao_feminina",      log_pos("populacao_feminina"))
    .withColumn("ln_Proporcao_pop_feminina",  log_pos("Proporcao_pop_feminina"))
    .withColumn("ln_eleitoras_mulheres",      log_pos("eleitoras_mulheres"))
    .withColumn("ln_IDHM_renda",              log_pos("IDHM_renda"))
    .withColumn("ln_IDHM_educacao",           log_pos("IDHM_educacao"))
    .withColumn("ln_IDHM_longevidade",        log_pos("IDHM_longevidade"))
    .withColumn("ln_prop_eleitoras_mulheres", log_pos("prop_eleitoras_mulheres"))
    .withColumn("ano", col("ano").cast("int"))
    .withColumn("id", F.concat_ws("_", col("sigla_uf"), col("cargo_std")))
    .withColumn(
        "ano_trat",
        F.when(col("cargo_std")=="vereador", lit(2012))
         .when(col("cargo_std").isin("deputado federal","deputado estadual","deputado distrital"), lit(2010))
         .otherwise(lit(0))
    )
    .orderBy("sigla_uf","cargo_std","ano")
)

display(panel_data)

DataFrame[sigla_uf: string, ano: int, cargo_std: string, n_total: bigint, n_candidatas: bigint, n_eleitos_total: bigint, n_eleitas: bigint, prop_candidatas: double, prop_eleitas: double, id_estado: string, id_cargo: string, tratado: int, treat_post: int, populacao_total: double, populacao_feminina: double, populacao_masculina: double, Proporcao_pop_feminina: double, Proporcao_pop_masculina: double, homicidios_total: double, feminicidios: double, eleitoras_mulheres: double, eleitores_homens: double, eleitores_sem_info: double, eleitores_total: double, IDHM: double, IDHM_renda: double, IDHM_educacao: double, IDHM_longevidade: double, prop_eleitoras_mulheres: double, ln_prop_eleitas: double, ln_prop_candidatas: double, ln_populacao_feminina: double, ln_Proporcao_pop_feminina: double, ln_eleitoras_mulheres: double, ln_IDHM_renda: double, ln_IDHM_educacao: double, ln_IDHM_longevidade: double, ln_prop_eleitoras_mulheres: double, id: string, ano_trat: int]

#### Validações rápidas

In [0]:
# (A) Uma linha por chave
dup = (panel_data.groupBy("sigla_uf","cargo_std","ano")
       .agg(F.count(F.lit(1)).alias("n")).filter(col("n")>1))
assert dup.count()==0, "Painel com duplicatas por sigla_uf×cargo_std×ano"

# (B) Faixas de proporção
mins_maxs = panel_data.select(
    F.min("prop_candidatas").alias("min_prop_candidatas"),
    F.max("prop_candidatas").alias("max_prop_candidatas"),
    F.min("prop_eleitas").alias("min_prop_eleitas"),
    F.max("prop_eleitas").alias("max_prop_eleitas")
).first().asDict()
print({"faixas_proporcoes": mins_maxs})

# (C) Dimensão esperada (se completamente balanceado)
n_ufs    = panel_data.select("sigla_uf").distinct().count()
n_cargos = panel_data.select("cargo_std").distinct().count()
n_anos   = panel_data.select("ano").distinct().count()
print({
    "linhas_painel": panel_data.count(),
    "n_ufs": n_ufs, "n_cargos": n_cargos, "n_anos": n_anos,
    "max_teorico_balanceado": n_ufs * n_cargos * n_anos
})

panel_data.write.mode("overwrite").saveAsTable("workspace.avaliacao_lei_tse.panel_data_ok")

{'faixas_proporcoes': {'min_prop_candidatas': 0.0, 'max_prop_candidatas': 0.75, 'min_prop_eleitas': 0.0, 'max_prop_eleitas': 1.0}}
{'linhas_painel': 4000, 'n_ufs': 27, 'n_cargos': 7, 'n_anos': 25, 'max_teorico_balanceado': 4725}


In [0]:
panel_data.write.mode("overwrite").saveAsTable("workspace.avaliacao_lei_tse.panel_data")

#### Preparar painel para rodar os modelos

In [0]:
# join painel + explicativas
panel_data_ok = panel_data.join(df_explicativas, on=["sigla_uf", "ano"], how="left")

INF = float("inf")
def is_finite(col):
    return (col.isNotNull()) & (~F.isnan(col)) & (F.abs(col) != F.lit(INF))

# janela para IDs sequenciais
w_rank = Window.orderBy("id_char")

# candidatas
df_did_cov_candidatas = (
    panel_data_ok
    .withColumn("id_char", F.concat_ws("_", "sigla_uf", "cargo_std"))
    .withColumn("id_num", F.dense_rank().over(w_rank).cast("int"))
    .withColumn("ln_prop_candidatas", F.log(F.col("prop_candidatas")))
    .withColumn("g", F.when(F.col("ano_trat").isNotNull(), F.col("ano_trat").cast("int")))
    .filter(is_finite(F.col("ln_prop_candidatas")))
    .filter(F.col("ano").isNotNull() & F.col("id_num").isNotNull() & F.col("g").isNotNull())
)
df_did_cov_candidatas_pd = df_did_cov_candidatas.toPandas()

# eleitas
df_did_cov_eleitas = (
    panel_data_ok
    .withColumn("id_char", F.concat_ws("_", "sigla_uf", "cargo_std"))
    .withColumn("id_num", F.dense_rank().over(w_rank).cast("int"))
    .withColumn("ln_prop_eleitas", F.log(F.col("prop_eleitas") + F.lit(0.01)))
    .withColumn("g", F.when(F.col("ano_trat").isNotNull(), F.col("ano_trat").cast("int")))
    .filter(is_finite(F.col("ln_prop_eleitas")))
    .filter(F.col("ano").isNotNull() & F.col("id_num").isNotNull() & F.col("g").isNotNull())
)
df_did_cov_eleitas_pd = df_did_cov_eleitas.toPandas()

# Deletar objetos da memoria
del cond_valido, df_candidatos, df_candidatos_all, df_candidatos_eleitos, df_candidatos_validos, \
    df_eleitos, df_genero_fix, df_genero_uf_fix, dup, grid, mapeamento_regioes, mapping_expr, \
    mins_maxs, painel_uf, painel_uf_bal, painel_uf_full, ufs_ruins, valid_ufs, w_back, w_fwd, \
    ano_max, ano_min, c, n_anos, n_cargos, n_ufs, anos_df, anos_todos, chaves, cond_genero_valido


/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(
/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(
/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(
/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data 

%md
# Estrutura do painel construído

- **Grão (chave primária):**
  - `sigla_uf` × `cargo_std` × `ano`

- **Período coberto:**
  - 1998–2022 (anos completados com LOCF entre pleitos)

- **Colunas principais:**
  - `sigla_uf`: unidade da federação (AC, AL, ..., SP, TO)
  - `cargo_std`: cargo padronizado ("vereador", "deputado estadual", "deputado federal", "deputado distrital", "senador", "governador", "prefeito")
  - `ano`: ano da eleição / painel
  - `n_total`: candidaturas totais válidas (M+F)
  - `n_candidatas`: candidaturas femininas
  - `prop_candidatas` = n_candidatas / n_total
  - `n_eleitos_total`: eleitos (M+F)
  - `n_eleitas`: eleitas
  - `prop_eleitas` = n_eleitas / n_eleitos_total
  - `tratado`: flag (1 se cargo é vereador ou deputado)
  - `treat_post`: flag (1 se ano >= 2012 para vereador, >= 2010 para deputados)
  - `prop_eleitoras_mulheres`: eleitoras_mulheres / eleitores_total
  - Variáveis socioeconômicas e logs (`ln_*`)

- **IDs do painel:**
  - `id_estado` = sigla_uf
  - `id_cargo` = cargo_std
  - `id` = concatenação textual → `sigla_uf + "_" + cargo_std`  
    - Exemplos:  
      - `"SP_vereador"`  
      - `"RJ_deputado estadual"`  
      - `"DF_deputado distrital"`

- **Opções de IDs adicionais (se necessário):**
  - `id_panel`: identificador numérico único para UF×cargo  
  - `id_panel_ano`: identificador numérico único para UF×cargo×ano  

- **Uso do `id`:**
  - `id` serve para identificar o grupo **UF×cargo**  
  - o tempo (`ano`) é mantido separado → chave completa é (`sigla_uf`, `cargo_std`, `ano`)  
  - facilita regressões com efeitos fixos (`fe = id`) e séries temporais (`ano`)


#### Exploração simples (resumos pré/pós lei por cargos e períodos)

In [0]:
def resumo_periodo(df, var, inicio, fim, cargos):
    return (
        df.loc[(df["ano"].between(inicio,fim)) & (df["cargo_std"].isin(cargos))]
          .groupby("ano")[var].mean()
          .agg(["min","max","mean"])
    )

# exemplo: deputados + vereadores
cargos_did = ["deputado federal","deputado estadual","deputado distrital","vereador"]
print("Pré-lei candidatas:", resumo_periodo(df_did_cov_candidatas_pd,"prop_candidatas",1996,2009,cargos_did))
print("Pós-lei candidatas:", resumo_periodo(df_did_cov_candidatas_pd,"prop_candidatas",2010,2022,cargos_did))


Pré-lei candidatas: min     0.161751
max     0.163525
mean    0.162281
Name: prop_candidatas, dtype: float64
Pós-lei candidatas: min     0.209829
max     0.352465
mean    0.291008
Name: prop_candidatas, dtype: float64


#### Resumo por cargo (para todos os cargos, pré e pós lei)


In [0]:
def resumo_por_cargo_all(df, var, inicio, fim, periodo):
    por_ano = (
        df.loc[df["ano"].between(inicio,fim)]
          .groupby(["cargo_std","ano"],as_index=False)[var].mean()
    )
    stats = por_ano.groupby("cargo_std")[var].agg(min="min",max="max",mean="mean").reset_index()
    stats["variavel"] = var
    stats["periodo"] = periodo
    return stats

cand_pre  = resumo_por_cargo_all(df_did_cov_candidatas_pd,"prop_candidatas",1996,2009,"Pré-lei")
cand_post = resumo_por_cargo_all(df_did_cov_candidatas_pd,"prop_candidatas",2010,2022,"Pós-lei")
ele_pre   = resumo_por_cargo_all(df_did_cov_eleitas_pd,"prop_eleitas",1996,2009,"Pré-lei")
ele_post  = resumo_por_cargo_all(df_did_cov_eleitas_pd,"prop_eleitas",2010,2022,"Pós-lei")

df_resumo = pd.concat([cand_pre,cand_post,ele_pre,ele_post],ignore_index=True)
display(df_resumo)


,cargo_std,min,max,mean,variavel,periodo
0,deputado distrital,0.204147,0.208075,0.205456,prop_candidatas,Pré-lei
1,deputado estadual,0.135095,0.139233,0.137854,prop_candidatas,Pré-lei
2,deputado federal,0.125097,0.134193,0.128129,prop_candidatas,Pré-lei
3,governador,0.188443,0.189547,0.188811,prop_candidatas,Pré-lei
4,prefeito,0.135367,0.147307,0.137357,prop_candidatas,Pré-lei
5,senador,0.186867,0.205239,0.192991,prop_candidatas,Pré-lei
6,vereador,0.219573,0.220702,0.220514,prop_candidatas,Pré-lei
7,deputado distrital,0.251880,0.352014,0.293634,prop_candidatas,Pós-lei
8,deputado estadual,0.207827,0.339204,0.276995,prop_candidatas,Pós-lei
9,deputado federal,0.200817,0.368229,0.283730,prop_candidatas,Pós-lei


### Plote da média de candidatas para os grupos tratado e controle

In [0]:
## Função para plotar 
def plot_parallel_trends(panel_pd: pd.DataFrame,
                         var: str = "prop_candidatas",
                         treated=("vereador", "deputado federal", "deputado estadual", "deputado distrital"),
                         controls=("senador", "governador", "prefeito"),
                         vline=2009,
                         title_prefix=""):
    # Checagens leves
    req_cols = {"ano", "cargo_std", var}
    missing = req_cols - set(panel_pd.columns)
    if missing:
        raise ValueError(f"Faltam colunas no DataFrame: {missing}")

    df = panel_pd.copy()
    # manter apenas linhas com ano e var válidos
    df = df.loc[df["ano"].notna() & df[var].notna()]

    # Definir grupo Tratado vs Controle
    df["grupo"] = np.where(df["cargo_std"].isin(treated), "Tratado",
                           np.where(df["cargo_std"].isin(controls), "Controle", "Outro"))

    # (opcional) filtrar só Tratado/Controle, como no R
    df = df[df["grupo"].isin(["Tratado", "Controle"])]

    # Agregar média por ano × grupo
    df_plot = (
        df.groupby(["ano", "grupo"], as_index=False)[var]
          .mean()
          .rename(columns={var: "media"})
    )

    # Ordenar por ano para garantir linhas contínuas
    df_plot = df_plot.sort_values(["grupo", "ano"])

    # Plot
    fig = px.line(
        df_plot,
        x="ano",
        y="media",
        color="grupo",
        markers=True,
        title=f"{title_prefix} – {var}"
    )

    # Formatação: eixos, hover e percentuais (var é proporção 0–1)
    fig.update_traces(mode="lines+markers", hovertemplate="%{x}<br>Média: %{y:.2%}<extra></extra>")
    fig.update_yaxes(title_text=f"Média de {var}", tickformat=".0%")
    fig.update_xaxes(title_text="Ano")

    # Linha vertical no ano da lei (2009)
    fig.add_vline(x=vline, line_width=2, line_dash="dash", line_color="red")
    fig.add_annotation(
        x=vline, y=1.02, xref="x", yref="paper",
        text=f"Lei Nº 12.034 / {vline}",
        showarrow=False, yanchor="bottom",
        font=dict(color="red")
    )

    # Layout geral
    fig.update_layout(
        legend_title_text="Grupo",
        template="simple_white",
        margin=dict(l=40, r=20, t=60, b=40),
        height=600 
    )

    return fig

# Gerar samploe
panel_pd = panel_data.select("ano", "cargo_std", "prop_candidatas", "prop_eleitas").toPandas()

In [0]:
display(panel_pd)

,ano,cargo_std,prop_candidatas,prop_eleitas
0,1998,deputado estadual,0.142012,0.083333
1,1999,deputado estadual,0.142012,0.083333
2,2000,deputado estadual,0.142012,0.083333
3,2001,deputado estadual,0.142012,0.083333
4,2002,deputado estadual,0.142012,0.083333
...,...,...,...,...
3995,2018,vereador,0.335921,0.155914
3996,2019,vereador,0.335921,0.155914
3997,2020,vereador,0.361334,0.174445
3998,2021,vereador,0.361334,0.174445


### Plote Proporção de candidatas

In [0]:
# Plotar prop_candidatas:
fig1 = plot_parallel_trends(panel_pd, var="prop_candidatas")
fig1



/databricks/python/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


### Plote Proporção de Eleitas

In [0]:
# Plotar prop_eleitas:
fig2 = plot_parallel_trends(panel_pd, var="prop_eleitas")
fig2

/databricks/python/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



## O que explica o aumento de mulheres eleitas a partir de 2017/2018?

---

### 1. Resolução TSE nº 23.553/2017  
Determinou que os partidos deveriam destinar **mínimo de 30% dos recursos do Fundo Partidário** para campanhas femininas.  
Além disso, obrigou que pelo menos **5% do fundo fosse aplicado em programas de promoção da participação política das mulheres**.  

### 2. Resolução TSE nº 23.575/2018  
Complementou a anterior, exigindo que **30% do Fundo Eleitoral (FEFC)** também fosse reservado às candidatas.  
> Pela primeira vez, a cota de gênero se traduziu em **financiamento real**, rompendo parcialmente a prática das “candidaturas laranjas” sem apoio.

---

### 3. Impacto observado em 2018  
- **290 mulheres eleitas** (16,2% do total), contra **190 em 2014** (11,1%).  
- Crescimento de **+52,6%** no número de mulheres eleitas em relação a 2014.  
- Na Câmara dos Deputados: **77 mulheres** eleitas (aumento de 51%).  
- Nas Assembleias Legislativas: **161 mulheres** (aumento de 41%).  

Fonte: [TSE – Número de mulheres eleitas em 2018 cresce 52,6% em relação a 2014](https://www.tse.jus.br/comunicacao/noticias/2019/Marco/numero-de-mulheres-eleitas-em-2018-cresce-52-6-em-relacao-a-2014)

---

### 4. Síntese
- O salto de 2018 não decorre de uma **nova lei** (a base segue sendo a Lei nº 12.034/2009).  
- Ele resulta da **aplicação efetiva de recursos financeiros vinculados às candidaturas femininas** via resoluções do TSE (2017/2018).  
- A diferença em relação ao período anterior é que, agora, as mulheres passaram a ter **financiamento garantido**, aumentando a competitividade e a taxa de sucesso eleitoral.

---

**Conclusão:** O crescimento de mulheres eleitas após 2017 se explica principalmente pelas resoluções do TSE que **destinaram obrigatoriamente recursos do fundo partidário e do fundo eleitoral às candidaturas femininas**. Foi a primeira vez que a cota de 30% deixou de ser apenas formal e passou a ter **lastro financeiro**, gerando impacto direto nos resultados de 2018.


### Preparar os dados para rodar um Causal Impact (package google)

In [0]:
treated = (
    panel_data
    .filter(panel_data.cargo_std.isin(["vereador","deputado federal","deputado estadual","deputado distrital"]))
    .groupBy("ano")
    .agg(F.mean("prop_candidatas").alias("prop_candidatas_tratado"))
    .orderBy("ano")
    .toPandas()
    .set_index("ano")
)

controls = (
    panel_data
    .filter(panel_data.cargo_std.isin(["senador","governador","prefeito"]))
    .groupBy("ano","cargo_std")
    .agg(F.mean("prop_candidatas").alias("prop_candidatas"))
    .toPandas()
    .pivot(index="ano", columns="cargo_std", values="prop_candidatas")
    .rename(columns={
        "senador": "prop_cand_senador",
        "governador": "prop_cand_governador",
        "prefeito": "prop_cand_prefeito"
    })
)

explicativas = (
    panel_data
    .groupBy("ano")
    .agg(
        F.mean("populacao_total").alias("populacao_total"),
        F.mean("populacao_feminina").alias("populacao_feminina"),
        F.mean("Proporcao_pop_feminina").alias("prop_pop_fem"),
        F.mean("eleitoras_mulheres").alias("eleitoras_mulheres"),
        F.mean("eleitores_total").alias("eleitores_total"),
        F.mean("IDHM").alias("idhm"),
        F.mean("IDHM_renda").alias("idhm_renda"),
        F.mean("IDHM_educacao").alias("idhm_educacao"),
        F.mean("IDHM_longevidade").alias("idhm_longevidade"),
        F.mean("homicidios_total").alias("homicidios_total"),
        F.mean("feminicidios").alias("feminicidios")
    )
    .orderBy("ano")
    .toPandas()
    .set_index("ano")
)

# salvar
df_ci_spark.write.mode("overwrite").saveAsTable("workspace.avaliacao_lei_tse.painel_causal_impact")

# visualizar
df_ci = treated.join(controls, how="left").join(explicativas, how="left")

### O Package causal impact será utilizado no google colab devido a imcompatibilidade com databricks